Edge cases:


0) Hərf
kəsmik-kesmik
yağ-yag

İnputdakı Hərflərin əvəzlənməsi

ü-u
ğ-g
ö-o
ş-sh
ə-e
ı-i

İstisnaya istina: Tualet kağızı və Tualet kagizi keysləri mövcuddur.

1) duz (başqa kateqoriya - yuyucu vasitə)
KAPLA VOX QABYUYAN MAYE DUZ EFFEKTI LIMON 1L

2) xiyar (başqa kateqoriya - turşu)
BIZIM TARLA XIYAR TURSU 670Q

3) mayonez (product name-də olmayan case)
MECHTA XOZYAYKI KLASSIK SHUSHE 900ML\

4) sosiska (product name əksərində sosiska yoxdur)

5) brokoli (BROCOLI yazılıb)

6) meyvə şirəsi (QAZSIZ ŞİRİN İÇKİLƏR)

In [ ]:
import re
import csv

def read_csv(file_path):
  products=[]
  with open(file_path,'r') as csvfile:
    reader=csv.DictReader(csvfile)
    for row in reader:
      price=float(row['price']) if row['price'] else None
      products.append({
          'name':row['name'],
          'price': price,
          'parent category': row['parent category'],
          'subcategory':row['subcategory'],
          'url':row['url']
      })
    return products

products=read_csv('/content/bravo-products-all.csv')
products

"""
ignore here
#def price_key(product):
#    return product['price']

or

import operator
...
products.sort(key=operator.itemgetter('price'))
ignore here
"""

# Function to get product recommendations based on category and strategy
def get_product_recommendation(data, name, strategy):
    # Define priority keywords and their corresponding categories
    priority_keywords = {
        'duz': 'ƏDVİYYATLAR',
        'xiyar': 'TƏRƏVƏZ'
    }

    if name.lower() in priority_keywords:
        # Filter products based on the corresponding category
        products = [item for item in data if item['subcategory'] == priority_keywords[name.lower()]]

        # Further filter products based on the keyword in their name
        products = [item for item in products if name.lower() in item['name'].lower()]

        # Filter out negative keywords for 'xiyar'
        if name.lower() == 'xiyar':
            negative_keywords = ['tursu', 'tursusu']
            products = [item for item in products if all(neg_kw not in item['name'].lower() for neg_kw in negative_keywords)]

    else:
        # If the keyword is not a priority keyword, search for products as before
        products = [item for item in data if name.lower() in item['name'].lower()]

    if strategy == 'ekonom':
        products.sort(key=lambda x: x['price'])
    elif strategy == 'premium':
        products.sort(key=lambda x: x['price'], reverse=True)

    return products[0] if products else None

# Function to process the user's shopping list
def process_shopping_list(data, budget, items, strategy):
    recommendations = []
    total_cost = 0
    for item_count, item_category in items:
        item_count = int(item_count)
        recommendation = get_product_recommendation(data, item_category, strategy)
        if not recommendation:
            print(f"{item_category} kateqoriyasında məhsul tapılmadı.")
            continue
        cost = recommendation['price'] * item_count
        while total_cost + cost > budget:
            print(f"{item_category} üçün büdcə çatmır.")
            required_budget = total_cost + cost - budget
            print(f"Əlavə olunmalı büdcə: {required_budget} azn")
            additional_budget = input("Əlavə büdcəni qeyd edin və ya SKIP yazın: ")
            if additional_budget.lower() == 'skip':
                break
            additional_budget = float(re.findall(r'\d+\.?\d*', additional_budget)[0])
            budget += additional_budget
            print(f"Yeni büdcə: {budget} azn")
            cost = recommendation['price'] * item_count
        else:
            total_cost += cost
            recommendations.append((item_count, recommendation))
    return recommendations, total_cost, budget #burda budget i qaytarırıq ki, update olunmuş dəyəri götürə bilək


def replace_characters(s):
    replacements = [
        ('ü', 'u'),
        ('ğ', 'g'),
        ('ö', 'o'),
        ('ş', 'sh'),
        ('ə', 'e'),
        ('ı', 'i'),
    ]

    for old, new in replacements:
        s = s.replace(old, new)
    return s

# Main function to handle user input and display results
def main():
    data = products  # Replace with the actual scraped data

    # Get user input for budget
    budget = input("zəhmət olmasa, büdcənizi qeyd edin: ")
    budget = float(re.findall(r'\d+\.?\d*', budget)[0])
    print(f"Büdcəniz {budget} azn təşkil edir.")

    # Get user input for shopping items
    items = input("zəhmət olmasa almaq istədiyiniz məhsulları prioritetinizə əsasən sayı ilə birlikdə ardıcıllıqla qeyd edin: ")

    # Prioritize matching user input with product name and category names before replacing characters
    original_items = items
    items = replace_characters(items)  # Replace characters in the user input

    # Use original_items for matching product name and category names
    items = re.findall(r'(\d+)\s*(?:ədəd|eded)?\s*([\w\s]+)', original_items) or re.findall(r'(\d+)\s*(?:ədəd|eded)?\s*([\w\s]+)', items)

    # Get user input for shopping strategy
    strategy = input("zəhmət olmasa, alış-veriş strategiyanızı seçin: ").lower()

    while True:
        # Process shopping list
        recommendations, total_cost, budget = process_shopping_list(data, budget, items, strategy) #funskiyanın qaytardığı dəyəri burdakı budget-ə assign edirik

        # Check if the budget is sufficient
        if total_cost <= budget:
            break
        else:
            print(f"Büdcəniz çatmır, zəhmət olmasa büdcənizi artırın.")
            print(f"Əlavə olunmalı büdcə: {total_cost - budget:.2f} azn")
            additional_budget = input("Əlavə büdcəni qeyd edin və ya SKIP yazın: ")
            if additional_budget.lower() == "skip":
                break
            additional_budget = float(re.findall(r'\d+\.?\d*', additional_budget)[0])
            budget += additional_budget
            print(f"Yeni büdcə: {budget} azn")

    # Display results
    if recommendations:
        print("Tövsiyə olunan məhsullar:")
        for count, item in recommendations:
            print(f"{item['name']} - {count} ədəd - {item['price']} azn")
        print(f"Ümumi məbləğ: {total_cost} azn")
        print(f"Qalıq büdcə: {budget - total_cost:.2f} azn")

# Check if the script is being run directly
if __name__ == "__main__":
    main()



zəhmət olmasa, büdcənizi qeyd edin: 10
Büdcəniz 10.0 azn təşkil edir.
zəhmət olmasa almaq istədiyiniz məhsulları prioritetinizə əsasən sayı ilə birlikdə ardıcıllıqla qeyd edin: 1 sacburan fen
zəhmət olmasa, alış-veriş strategiyanızı seçin: ekonom
sacburan fen üçün büdcə çatmır.
Əlavə olunmalı büdcə: 19.99 azn
Əlavə büdcəni qeyd edin və ya SKIP yazın: 20
Yeni büdcə: 30.0 azn
Tövsiyə olunan məhsullar:
SACBURAN FEN ROWENTA CF2132 - 1 ədəd - 29.99 azn
Ümumi məbləğ: 29.99 azn
Qalıq büdcə: 0.01 azn
